In [2]:
import pandas as pd

path = r'C:\Users\Levy\Desktop\ICA\Dados\datasets\artificial\batman\data.csv'
data = pd.read_csv(path)

data = pd.DataFrame(data)

In [8]:
data['2'].value_counts()

2
1    6635
2    3364
Name: count, dtype: int64

In [13]:
# NOrmalizando os dados
from sklearn.preprocessing import MinMaxScaler

normalizador = MinMaxScaler()
normalizador.fit(data)

dados_normalizados = pd.DataFrame(normalizador.transform(data), columns = data.columns)
dados_normalizados.head()

X = dados_normalizados.drop(columns='2')
y = dados_normalizados['2']


In [14]:
# Separando os dados para o treinamento
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=2000, solver='adam')

mlp.fit(X_train, y_train)

predi = mlp.predict(X_test)

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predi)

# confusion_matrix(y_test, predi)[1]

array([[1257,  106],
       [ 111,  526]], dtype=int64)

In [28]:
sensibilidade = confusion_matrix(y_test, predi)[0][0]/(confusion_matrix(y_test, predi)[0][0] + confusion_matrix(y_test, predi)[0][1])
especificidade = confusion_matrix(y_test, predi)[1][1]/(confusion_matrix(y_test, predi)[1][0] + confusion_matrix(y_test, predi)[1][1])
acuracia = (confusion_matrix(y_test, predi)[1][1] + confusion_matrix(y_test, predi)[0][0])/(confusion_matrix(y_test, predi)[0][0] + confusion_matrix(y_test, predi)[1][1] + confusion_matrix(y_test, predi)[1][0] + confusion_matrix(y_test, predi)[0][1])

print(sensibilidade, especificidade, acuracia)


0.9222303741746148 0.82574568288854 0.8915


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

specifity = classification_report(y_test, predi)
recall = recall_score(y_test, predi)
acc = accuracy_score(y_test, predi)

print(f'acc: {acc}, recall: {recall}, specifity: {specifity}')

acc: 0.8915, recall: 0.82574568288854, specifity:               precision    recall  f1-score   support

         0.0       0.92      0.92      0.92      1363
         1.0       0.83      0.83      0.83       637

    accuracy                           0.89      2000
   macro avg       0.88      0.87      0.87      2000
weighted avg       0.89      0.89      0.89      2000

